In [1]:
## Making the taxi environment to train a RL agent in

In [2]:
import gym

env = gym.make("Taxi-v3")

In [3]:
env.reset()

print (f"Observation Space: {env.observation_space}.")
print (f"Action Space: {env.action_space}.")

env.render()

Observation Space: Discrete(500).
Action Space: Discrete(6).
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [4]:
env.reset()
env.s = 328

epochs = 0
penalties, reward = 0, 0

frames = []

done = False

while not done:
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)
    
    if reward == -10:
        penalties += 1
        
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
    })
    
    epochs += 1
    
print (f"Timesteps taken: {epochs}")
print (f"Penalties incurred: {penalties}")

Timesteps taken: 200
Penalties incurred: 67


In [5]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame.get('frame'))
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.1)
        
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)

Timestep: 200
State: 414
Action: 2
Reward: -1


In [6]:
# Introducing the Q-learning agent to the environment

In [7]:
# The basic steps of a Q-learning agent goes
# 1. Initialize a Q-table
# 2. Explore: for each state, select any action among all states
# 3. Travel to next state (s') using (a)
# 4. From state (s') select action with highest Q-value
# 5. Update Q-values using the equation
# 6. Update (s) with (s')
# 7. If goal is reached, then end and do next episode

In [8]:
## Initialize Q-table
import numpy as np

q_table = np.zeros([env.observation_space.n, env.action_space.n])

In [9]:
q_table.shape

(500, 6)

In [10]:
# No agent
# Evaluation
total_epochs, total_penalties = 0, 0
episodes = 100

for _ in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")


Results after 100 episodes:
Average timesteps per episode: 200.0
Average penalties per episode: 0.0


In [11]:
## The agent
import random
from IPython.display import clear_output

# Hyperparameters
alpha = 0.1 # Learning Rate: ratio of new value updating old value
gamma = 0.6 # Discount: How important is the evaluation of the next timestep
epsilon = 0.1 # Random exploration

# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, 100001):
    state = env.reset()
    
    epochs, penalties, reward = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0,1) < epsilon:
            action = env.action_space.sample()
        else:
            action = np.argmax(q_table[state])
            
        next_state, reward, done, info = env.step(action)
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1-alpha) * old_value + alpha*(reward+ gamma*next_max)
        q_table[state,action] = new_value
        
        if reward == -10:
            penalties += 1
            
        state = next_state
        epochs += 1
        
    if i%100 == 0:
        clear_output(wait=True)
        print (f"Episode:{i}")
        
print ("Finished training.")

Episode:100000
Finished training.


In [12]:
# Evaluate agent
total_epochs, total_penalties = 0, 0
episodes = 100

for _ in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")


Results after 100 episodes:
Average timesteps per episode: 13.01
Average penalties per episode: 0.0
